# (II) Deteccion y Picking
Este cuaderno demuestra el uso de EQTransformer para realizar la detección de señales sísmicas y la selección de fases sísmicas (P y S) en datos continuos. Una vez que tengas tus datos sísmicos —preferentemente en formato MiniSeed y organizados en subcarpetas individuales para cada estación— puedes realizar la detección y selección usando las siguientes opciones:

### Opción (I) con archivos preprocesados (hdf5):

Esta opción es recomendada para períodos cortos de tiempo (desde algunos días hasta un mes). Permite probar el desempeño y explorar los efectos de diferentes parámetros, mientras que el archivo hdf5 facilita el acceso a las formas de onda.

Para esta opción primero necesitas convertir tus archivos MiniSeed de cada estación en un único archivo hdf5 y un archivo csv que contenga la lista de trazas dentro del archivo hdf5.

Puedes convertir los archivos MiniSeed a un archivo hdf5 usando el siguiente comando:

In [1]:
import os
from EQTransformer.utils.hdf5_maker import preprocessor

json_basepath = os.path.join(os.getcwd(),"json/station_list.json")

preprocessor(preproc_dir="preproc",
             mseed_dir='downloads_mseeds', 
             stations_json=json_basepath, 
             overlap=0.3, 
             n_processor=2)

============ Station SV08 has 2 chunks of data.
============ Station CA06 has 2 chunks of data.
  * CA06 (1) .. 20190901 --> 20190902 .. 3 components .. sampling rate: 100.0
  * SV08 (1) .. 20190901 --> 20190902 .. 3 components .. sampling rate: 100.0
  * CA06 (2) .. 20190902 --> 20190903 .. 3 components .. sampling rate: 100.0
  * SV08 (2) .. 20190902 --> 20190903 .. 3 components .. sampling rate: 100.0
 Station CA06 had 2 chuncks of data
4112 slices were written, 4114.0 were expected.
Number of 1-components: 0. Number of 2-components: 0. Number of 3-components: 2.
Original samplieng rate: 100.0.
============ Station B921 has 2 chunks of data.
  * B921 (1) .. 20190901 --> 20190902 .. 3 components .. sampling rate: 100.0
 Station SV08 had 2 chuncks of data
4112 slices were written, 4114.0 were expected.
Number of 1-components: 0. Number of 2-components: 0. Number of 3-components: 2.
Original samplieng rate: 100.0.
  * B921 (2) .. 20190902 --> 20190903 .. 3 components .. sampling rate: 

Esto generará un archivo "nombre_estación.hdf5" y un archivo "nombre_estación.csv" para cada estación, y los colocará en un directorio llamado "mseed_dir+_hdfs". Luego debes pasar el nombre del directorio que contiene tus archivos hdf5 y csv junto con un modelo. Puedes usar valores de umbral relativamente bajos para la detección y selección, ya que EQTransformer es muy robusto frente a falsos positivos. Activar la estimación de incertidumbre, producir probabilidades o graficar todos los eventos detectados ralentizará el proceso.

In [2]:
from EQTransformer.core.predictor import predictor
predictor(input_dir='downloads_mseeds_processed_hdfs',   
         input_model='../ModelsAndSampleData/EqT_original_model.h5',
         output_dir='detections1',
         estimate_uncertainty=False, 
         output_probabilities=False,
         number_of_sampling=5,
         loss_weights=[0.02, 0.40, 0.58],          
         detection_threshold=0.3,                
         P_threshold=0.3,
         S_threshold=0.3, 
         number_of_plots=10,
         plot_mode='time',
         batch_size=500,
         number_of_cpus=4,
         keepPS=False,
         spLimit=60) 

Running EqTransformer  0.1.58
 *** Loading the model ...
*** Loading is complete!
######### There are files for 3 stations in downloads_mseeds_processed_hdfs directory. #########
========= Started working on B921, 1 out of 3 ...
100%|█████████████████████████████████████████████████████████████████| 9/9 [01:21<00:00,  9.01s/it]WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.


 *** Finished the prediction in: 0 hours and 1 minutes and 24.73 seconds.
 *** Detected: 5376 events.
 *** Wrote the results into --> " /home/santiago/Coding University/PFC-TRANSFORMER-EQ/examples/detections1/B921_outputs "
========= Started working on CA06, 2 out of 3 ...
100%|█████████████████████████████████████████████████████████████████| 9/9 [01:24<00:00,  9.42s/it]


 *** Finished the prediction in: 0 hours and 1 minutes and 14.89 seconds.
 *** Detected: 5422 events.
 *** Wrote the results 

Las salidas de la predicción para cada estación se escribirán en tu directorio de salida (por ejemplo, 'detections').

El archivo 'X_report.txt' contiene información sobre los parámetros de entrada usados para la detección/selección y los resultados finales, como el tiempo de ejecución y el número total de eventos detectados (estos son eventos únicos, ya que se eliminaron duplicados).

El archivo 'X_prediction_results.csv' contiene los resultados de detección/selección, y en la carpeta de figuras encontrarás los gráficos para el número de eventos que especificaste en el comentario anterior.